In [1]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.initializers import RandomNormal, GlorotNormal
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects

def simplenet(act, s = 2):
    model = Sequential()

    model.add(Conv2D(64, (3,3), padding='same', kernel_initializer=GlorotNormal(), input_shape=(32, 32, 3)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=RandomNormal(stddev=0.01)))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    
    model.add(Conv2D(512, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(BatchNormalization())
    model.add(Activation(act))
    model.add(Dropout(0.2))

    model.add(Conv2D(2048, (1,1), padding='same', kernel_initializer=GlorotNormal()))
    model.add(Activation(act))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256, (1,1), padding='same', kernel_initializer=GlorotNormal()))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, (3,3), padding='same', kernel_initializer=GlorotNormal()))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2), strides=s))

    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))

    return model

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = simplenet('gelu')

batch_size = 128
epochs = 50

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)


print(history.history)
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test,axis=1)

print(y_pred.shape)
print(y_true.shape)

print(np.sum(y_pred == y_true) / y_pred.shape[0])

170508288/170498071 [==============================] - 6s 0us/step
Epoch 1/50
313/313 [==============================] - 168s 365ms/step - loss: 2.1498 - accuracy: 0.1807 - val_loss: 3.1431 - val_accuracy: 0.0976
Epoch 2/50
313/313 [==============================] - 112s 357ms/step - loss: 1.6971 - accuracy: 0.3418 - val_loss: 3.3563 - val_accuracy: 0.1006
Epoch 3/50
313/313 [==============================] - 112s 357ms/step - loss: 1.5324 - accuracy: 0.4165 - val_loss: 4.3640 - val_accuracy: 0.1299
Epoch 4/50
313/313 [==============================] - 112s 357ms/step - loss: 1.3748 - accuracy: 0.4861 - val_loss: 4.1633 - val_accuracy: 0.1498
Epoch 5/50
313/313 [==============================] - 112s 357ms/step - loss: 1.2255 - accuracy: 0.5484 - val_loss: 3.7204 - val_accuracy: 0.1600
Epoch 6/50
313/313 [==============================] - 112s 357ms/step - loss: 1.1030 - accuracy: 0.6003 - val_loss: 2.8878 - val_accuracy: 0.2446
Epoch 7/50
313/313 [==============================] - 112